In [1]:
from SPEP import spep
import os
import numpy as np

In [2]:
# Selecting mask and reference images
ref_name = 'datasets/dataset_1/A/A-kappa/Z_80080808_0230_IFIX1.jpg'
mask_name = 'datasets/dataset_1/A/A-kappa/Z_80090997_0230_IFIX1.jpg'
directory = 'datasets'

In [3]:
ref = spep(ref_name)
mask = spep(mask_name)

spep.albumin_reference(mask)
spep.barLenght_reference(ref)
spep.bar_finder(ref)

In [4]:
count_all = 0
count_valid = 0
distance_ratio = {}
distance = {}
lines = {"albumin": [], "alpha1": [], "alpha2": [], "beta": [], "gamma": []}

In [5]:
for root, dir, files in os.walk(directory):
    for file in files:
        file_dir = os.path.join(root,file)

        if file_dir.endswith('.jpg'):
            count_all += 1

            obj = spep(file_dir)

            obj.evaluate()
            obj.find_lines(['all'],smoothRange= 5, dentRange= 10, use_combinations= True)
            
            if len(obj.lines) == 5:
                count_valid += 1

                for i in range(5):
                    obj.lines[i] = 100*(obj.lines[i]/spep.bar) # Normalization

                lines["albumin"].append(obj.lines[0])
                lines["alpha1"].append(obj.lines[1])
                lines["alpha2"].append(obj.lines[2])
                lines["beta"].append(obj.lines[3])  
                lines["gamma"].append(obj.lines[4])

                obj.line_dist()
                for i,(key1,value1) in enumerate(obj.dist_dict.items()):
                    for j,(key2,value2) in enumerate(obj.dist_dict.items()):
                        if i>=j:
                            continue
                        else:
                            if f'{key1}:{key2}' in distance_ratio:
                                distance_ratio[f'{key1}:{key2}'].append(value1/value2)
                            else:
                                distance_ratio[f'{key1}:{key2}'] =[]
                                distance_ratio[f'{key1}:{key2}'].append(value1/value2)

                for key,value in obj.dist_dict.items():
                    if key in distance:
                        distance[key].append(value)
                    else:
                        distance[key] = list()
                        distance[key].append(value)

In [6]:
for k, v in lines.items():
    print(f"{k} mean: {np.mean(v):.2f}\n{k} std: {np.std(v):.2f}\n----------------------")

albumin mean: 10.22
albumin std: 6.97
----------------------
alpha1 mean: 27.86
alpha1 std: 7.91
----------------------
alpha2 mean: 47.46
alpha2 std: 8.00
----------------------
beta mean: 62.89
beta std: 7.26
----------------------
gamma mean: 83.98
gamma std: 6.72
----------------------


In [7]:
# Writing the findings
print('#images: ', count_all)
print('valid images: ',round(100*count_valid/count_all,2),'%')
print('bar lenght: ',spep.bar)
print('-----------------------')
for key, value in distance.items():
    print(f'{key} mean: {np.mean(value):.2f}\n{key} std: {np.std(value):.2f}\n-----------------------')

#images:  1778
valid images:  98.26 %
bar lenght:  184
-----------------------
albumin - alpha1 mean: 17.64
albumin - alpha1 std: 5.39
-----------------------
albumin - alpha2 mean: 37.24
albumin - alpha2 std: 7.89
-----------------------
albumin - beta mean: 52.66
albumin - beta std: 7.81
-----------------------
albumin - gamma mean: 73.76
albumin - gamma std: 8.93
-----------------------
alpha1 - alpha2 mean: 19.60
alpha1 - alpha2 std: 6.20
-----------------------
alpha1 - beta mean: 35.02
alpha1 - beta std: 7.17
-----------------------
alpha1 - gamma mean: 56.12
alpha1 - gamma std: 9.13
-----------------------
alpha2 - beta mean: 15.43
alpha2 - beta std: 5.49
-----------------------
alpha2 - gamma mean: 36.52
alpha2 - gamma std: 8.02
-----------------------
beta - gamma mean: 21.09
beta - gamma std: 6.59
-----------------------


In [8]:
for key,value in distance_ratio.items():
    print(f'{key} ratio mean: {np.mean(value):.2f}')
    print(f'{key} ratio std: {np.std(value):.2f}\n--------------------')

albumin - alpha1:albumin - alpha2 ratio mean: 0.48
albumin - alpha1:albumin - alpha2 ratio std: 0.11
--------------------
albumin - alpha1:albumin - beta ratio mean: 0.34
albumin - alpha1:albumin - beta ratio std: 0.09
--------------------
albumin - alpha1:albumin - gamma ratio mean: 0.24
albumin - alpha1:albumin - gamma ratio std: 0.07
--------------------
albumin - alpha1:alpha1 - alpha2 ratio mean: 1.02
albumin - alpha1:alpha1 - alpha2 ratio std: 0.59
--------------------
albumin - alpha1:alpha1 - beta ratio mean: 0.54
albumin - alpha1:alpha1 - beta ratio std: 0.26
--------------------
albumin - alpha1:alpha1 - gamma ratio mean: 0.33
albumin - alpha1:alpha1 - gamma ratio std: 0.16
--------------------
albumin - alpha1:alpha2 - beta ratio mean: 1.29
albumin - alpha1:alpha2 - beta ratio std: 0.67
--------------------
albumin - alpha1:alpha2 - gamma ratio mean: 0.52
albumin - alpha1:alpha2 - gamma ratio std: 0.28
--------------------
albumin - alpha1:beta - gamma ratio mean: 0.97
album

In [9]:
distance_ratio_std = {}
for key,value in distance_ratio.items():
    distance_ratio_std[key] = np.std(value)
sorted_dict = {k: v for k, v in sorted(distance_ratio_std.items(), key=lambda item: item[1])}
for key, value in sorted_dict.items():
    print(f'{key} std: {value:.3f}')

albumin - alpha1:albumin - gamma std: 0.073
albumin - beta:albumin - gamma std: 0.077
albumin - alpha1:albumin - beta std: 0.088
alpha1 - beta:alpha1 - gamma std: 0.091
albumin - alpha2:albumin - gamma std: 0.091
alpha1 - alpha2:alpha1 - gamma std: 0.093
albumin - alpha2:albumin - beta std: 0.096
albumin - alpha1:albumin - alpha2 std: 0.107
alpha2 - beta:alpha2 - gamma std: 0.113
alpha1 - alpha2:alpha1 - beta std: 0.117
albumin - alpha1:alpha1 - gamma std: 0.159
albumin - gamma:alpha1 - gamma std: 0.159
albumin - alpha2:alpha1 - gamma std: 0.191
albumin - beta:alpha1 - gamma std: 0.191
albumin - alpha1:alpha1 - beta std: 0.260
albumin - beta:alpha1 - beta std: 0.260
albumin - alpha1:alpha2 - gamma std: 0.277
alpha1 - alpha2:alpha2 - gamma std: 0.278
alpha1 - gamma:alpha2 - gamma std: 0.278
albumin - alpha2:alpha1 - beta std: 0.288
alpha1 - beta:alpha2 - gamma std: 0.308
albumin - gamma:alpha1 - beta std: 0.414
albumin - alpha2:alpha2 - gamma std: 0.453
albumin - gamma:alpha2 - gamma st

# Single Method

## Sharp

In [10]:
count_valid = 0
count_all = 0
for root, dir, files in os.walk(directory):
    for file in files:
        file_dir = os.path.join(root,file)

        if file_dir.endswith('.jpg'):
            count_all += 1

            obj = spep(file_dir)

            obj.evaluate()
            obj.find_lines(['sharp'],smoothRange= 5, dentRange= 10, increase_spec= False)
            
            if len(obj.lines) == 5:
                count_valid += 1
print(f'Sharp method validation: {100*count_valid/count_all:.2f}%')

Sharp method validation: 16.03%


## Sharp v2

In [11]:
count_valid = 0
count_all = 0
for root, dir, files in os.walk(directory):
    for file in files:
        file_dir = os.path.join(root,file)

        if file_dir.endswith('.jpg'):
            count_all += 1

            obj = spep(file_dir)

            obj.evaluate()
            obj.find_lines(['sharp_v2'],smoothRange= 5, dentRange= 10, increase_spec= False)
            
            if len(obj.lines) == 5:
                count_valid += 1
print(f'Sharp_v2 method validation: {100*count_valid/count_all:.2f}%')

Sharp_v2 method validation: 5.46%


## Sharp v3

In [12]:
count_valid = 0
count_all = 0
for root, dir, files in os.walk(directory):
    for file in files:
        file_dir = os.path.join(root,file)

        if file_dir.endswith('.jpg'):
            count_all += 1

            obj = spep(file_dir)

            obj.evaluate()
            obj.find_lines(['sharp_v3'],smoothRange= 5, dentRange= 10, increase_spec= False)
            
            if len(obj.lines) == 5:
                count_valid += 1
print(f'Sharp_v3 method validation: {100*count_valid/count_all:.2f}%')

Sharp_v3 method validation: 6.36%


## Smooth

In [13]:
count_valid = 0
count_all = 0
for root, dir, files in os.walk(directory):
    for file in files:
        file_dir = os.path.join(root,file)

        if file_dir.endswith('.jpg'):
            count_all += 1

            obj = spep(file_dir)

            obj.evaluate()
            obj.find_lines(['smooth'],smoothRange= 5, dentRange= 10, increase_spec= False)
            
            if len(obj.lines) == 5:
                count_valid += 1
print(f'Smooth method validation: {100*count_valid/count_all:.2f}%')

Smooth method validation: 0.67%


## Dent

In [14]:
count_valid = 0
count_all = 0
for root, dir, files in os.walk(directory):
    for file in files:
        file_dir = os.path.join(root,file)

        if file_dir.endswith('.jpg'):
            count_all += 1

            obj = spep(file_dir)

            obj.evaluate()
            obj.find_lines(['dent'],smoothRange= 5, dentRange= 10, increase_spec= False)
            
            if len(obj.lines) == 5:
                count_valid += 1
print(f'Dent method validation: {100*count_valid/count_all:.2f}%')

# Smooth-Dent

In [ ]:
count_valid = 0
count_all = 0
for root, dir, files in os.walk(directory):
    for file in files:
        file_dir = os.path.join(root,file)

        if file_dir.endswith('.jpg'):
            count_all += 1

            obj = spep(file_dir)

            obj.evaluate()
            obj.find_lines(['smooth','dent'],smoothRange= 5, dentRange= 10, increase_spec= False)
            
            if len(obj.lines) == 5:
                count_valid += 1
print(f'Dent method validation: {100*count_valid/count_all:.2f}%')

Dent method validation: 77.39%
